## Tool Calling

In this section, we'll explore how to enhance your applications with tool calling capabilities. We'll cover:
1. Setting up and using the Brave Search API
2. Creating custom tools
3. Configuring tool prompts and safety settings

In [3]:
!pip install llama-stack-client --upgrade

In [4]:
import asyncio
import os
from typing import Dict, List, Optional
from dotenv import load_dotenv

from llama_stack_client import LlamaStackClient
#from llama_stack_client.lib.agents.agent import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.types.agent_create_params import (
    AgentConfig,
    AgentConfigToolSearchToolDefinition,
)

# Load environment variables
load_dotenv()

# Helper function to create an agent with tools
async def create_tool_agent(
    client: LlamaStackClient,
    tools: List[Dict],
    instructions: str = "You are a helpful assistant",
    model: str = "Llama3.1-8B-Instruct",
) -> Agent:
    """Create an agent with specified tools."""
    agent_config = AgentConfig(
        model=model,
        instructions=instructions,
        sampling_params={
            "strategy": "greedy",
            "temperature": 1.0,
            "top_p": 0.9,
        },
        tools=tools,
        tool_choice="auto",
        tool_prompt_format="json",
        input_shields=["llama_guard"],
        output_shields=["llama_guard"],
        enable_session_persistence=True,
    )

    return Agent(client, agent_config)

NameError: name 'Agent' is not defined

First, create a `.env` file in your notebook directory with your Brave Search API key:

```
BRAVE_SEARCH_API_KEY=your_key_here
```


In [ ]:
async def create_search_agent(client: LlamaStackClient) -> Agent:
    """Create an agent with Brave Search capability."""
    search_tool = AgentConfigToolSearchToolDefinition(
        type="brave_search",
        engine="brave",
        api_key=os.getenv("BRAVE_SEARCH_API_KEY"),
    )

    return await create_tool_agent(
        client=client,
        tools=[search_tool],
        instructions="""
        You are a research assistant that can search the web.
        Always cite your sources with URLs when providing information.
        Format your responses as:

        FINDINGS:
        [Your summary here]

        SOURCES:
        - [Source title](URL)
        """
    )

# Example usage
async def search_example():
    client = LlamaStackClient(base_url="http://localhost:8000")
    agent = await create_search_agent(client)

    # Create a session
    session_id = agent.create_session("search-session")

    # Example queries
    queries = [
        "What are the latest developments in quantum computing?",
        "Who won the most recent Super Bowl?",
    ]

    for query in queries:
        print(f"\nQuery: {query}")
        print("-" * 50)

        response = agent.create_turn(
            messages=[{"role": "user", "content": query}],
            session_id=session_id,
        )

        async for log in EventLogger().log(response):
            log.print()

# Run the example (in Jupyter, use asyncio.run())
await search_example()

## 3. Custom Tool Creation

Let's create a custom weather tool:

In [ ]:
from typing import TypedDict, Optional
from datetime import datetime

# Define tool types
class WeatherInput(TypedDict):
    location: str
    date: Optional[str]

class WeatherOutput(TypedDict):
    temperature: float
    conditions: str
    humidity: float

class WeatherTool:
    """Example custom tool for weather information."""

    def __init__(self, api_key: Optional[str] = None):
        self.api_key = api_key

    async def get_weather(self, location: str, date: Optional[str] = None) -> WeatherOutput:
        """Simulate getting weather data (replace with actual API call)."""
        # Mock implementation
        return {
            "temperature": 72.5,
            "conditions": "partly cloudy",
            "humidity": 65.0
        }

    async def __call__(self, input_data: WeatherInput) -> WeatherOutput:
        """Make the tool callable with structured input."""
        return await self.get_weather(
            location=input_data["location"],
            date=input_data.get("date")
        )

async def create_weather_agent(client: LlamaStackClient) -> Agent:
    """Create an agent with weather tool capability."""
    weather_tool = {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get weather information for a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "City or location name"
                    },
                    "date": {
                        "type": "string",
                        "description": "Optional date (YYYY-MM-DD)",
                        "format": "date"
                    }
                },
                "required": ["location"]
            }
        },
        "implementation": WeatherTool()
    }

    return await create_tool_agent(
        client=client,
        tools=[weather_tool],
        instructions="""
        You are a weather assistant that can provide weather information.
        Always specify the location clearly in your responses.
        Include both temperature and conditions in your summaries.
        """
    )

# Example usage
async def weather_example():
    client = LlamaStackClient(base_url="http://localhost:8000")
    agent = await create_weather_agent(client)

    session_id = agent.create_session("weather-session")

    queries = [
        "What's the weather like in San Francisco?",
        "Tell me the weather in Tokyo tomorrow",
    ]

    for query in queries:
        print(f"\nQuery: {query}")
        print("-" * 50)

        response = agent.create_turn(
            messages=[{"role": "user", "content": query}],
            session_id=session_id,
        )

        async for log in EventLogger().log(response):
            log.print()

# Run the example
await weather_example()